<a href="https://colab.research.google.com/github/eduardez/BBDD_Avanzadas_Datawarehouse/blob/master/Manejo_de_BBDD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Manejo de datos

In [1]:
import pandas as pd
import numpy as np


## Carga de datos

In [2]:
AESO_url  = 'https://raw.githubusercontent.com/eduardez/BBDD_Avanzadas_Datawarehouse/CSV/ETEFIL_AESO_2019.csv'
ESO_url  = 'https://raw.githubusercontent.com/eduardez/BBDD_Avanzadas_Datawarehouse/CSV/ETEFIL_ESO_2019.csv'
BACH_url  = 'https://raw.githubusercontent.com/eduardez/BBDD_Avanzadas_Datawarehouse/CSV/ETEFIL_BACH_2019.csv'
GM_url  = 'https://raw.githubusercontent.com/eduardez/BBDD_Avanzadas_Datawarehouse/CSV/ETEFIL_GM_2019.csv'
GS_url  = 'https://raw.githubusercontent.com/eduardez/BBDD_Avanzadas_Datawarehouse/CSV/ETEFIL_GS_2019.csv'

AESO_dataset = pd.read_csv(AESO_url, index_col=[0], delimiter='\t')
ESO_dataset = pd.read_csv(ESO_url, index_col=[0], delimiter='\t')
BACH_dataset = pd.read_csv(BACH_url, index_col=[0], delimiter='\t')
GM_dataset = pd.read_csv(GM_url, index_col=[0], delimiter='\t')
GS_dataset = pd.read_csv(GS_url, index_col=[0], delimiter='\t')

FULL_dataset = {
    'Abandono_ESO':AESO_dataset,
    'ESO':ESO_dataset,
    'BACH':BACH_dataset,
    'GradoMedio':GM_dataset,
    'GradoSuperior':GS_dataset,
}

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (113) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
AESO_dataset.head()

,SEXO,ANIO_NAC,NACIO,NACIO1,NACIO2,PAIS_NACI,T_CENTRO,CCAA,DISCA,PROXY,LUG_RES_PAIS,CA_RES,PAIS_NAC_PADRE,ESTUDIOS_PADRE,PAIS_NAC_MADRE,ESTUDIOS_MADRE,ESTA_B2,ESTA_B4,ESTA_B5,ESTA_B6,ESTA_B7,EST_B8,EST_B9,EST_B10,EST_B11,EST_B12A,EST_B12B_1,EST_B15,EST_B16,ESTA_B17,ESTA_B18,ESTA_B19,ESTA_B20,EST_B21,EST_B23A,EST_B23B_1,EST_B23B_2,EST_B25,EST_B27,EST_B28,...,DIAS16,BCOTC16,ALSSM17,COTIZ17,DIAS17,BCOTC17,ALSSM18,COTIZ18,DIAS18,BCOTC18,ALSSM19,COTIZ19,DIAS19,BCOTC19,N_CURS,DENCUR1,DURCUR1,RESCUR1,ANHCUR1,DENCUR2,DURCUR2,RESCUR2,ANHCUR2,DENCUR3,DURCUR3,RESCUR3,ANHCUR3,DENCUR4,DURCUR4,RESCUR4,ANHCUR4,DENCUR5,DURCUR5,RESCUR5,ANHCUR5,DENCUR6,DURCUR6,RESCUR6,ANHCUR6,FACTOR
IDENT,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1,1995,1,,,1,1,3,2,1,1,3,1,4,3,4,4,2,1,5,4,,,,,,,,,,,,,,,,,,,,...,NaN,,2,,NaN,,2,,NaN,,2,,NaN,,0,,,,,,,,,,,,,,,,,,,,,,,,,25.643750
2,2,1995,1,,,1,1,1,1,1,1,1,1,1,1,4,4,8,2,,,,,,,,,,,,,,,,,,,,,,...,NaN,,2,,NaN,,2,,NaN,,2,,NaN,,0,,,,,,,,,,,,,,,,,,,,,,,,,82.244755
3,1,1996,1,,,3,1,8,2,1,1,8,1,3,1,2,4,2,1,1,5,,,,,,,,,,,,,,,,,,,,...,NaN,,2,,NaN,,2,,NaN,,2,,NaN,,0,,,,,,,,,,,,,,,,,,,,,,,,,40.030361
4,1,1996,1,,,1,1,8,2,1,1,8,1,2,1,2,4,9,1,1,4,,,,,,,,,,,,,,,,,,,,...,31.0,2,1,2,31.0,2,1,2,31.0,4,1,2,31.0,4,0,,,,,,,,,,,,,,,,,,,,,,,,,40.030361
5,2,1996,1,,,1,1,1,2,1,1,1,1,4,1,4,4,4,1,1,4,,,,,,,,,,,,,,,,,,,,...,NaN,,2,,NaN,,2,,NaN,,1,2,31.0,1,0,,,,,,,,,,,,,,,,,,,,,,,,,82.244755


###Comprobar valores nulos

Como en los dataset originales los valores nulos estaban representados mediante un espacio o mediante NaN, hay que sustituirlos que son espacios por NaN para asi poder sacar el porcentaje verdadero de nulos y si es mayor que el umbral que hayamos establecido, esas columnas se eliminan.

In [4]:
def getNulos(df):
  porcentaje_nulos = df.isnull().sum() * 100 / len(df)
  missing_value_df = pd.DataFrame({'column_name': df.columns,
                                  'percent_missing': porcentaje_nulos})
  missing_value_df.sort_values('percent_missing', inplace=True)
  return missing_value_df

def getFULLNulos(arr_df):
  nulos_dict = {}
  for key_df in FULL_dataset:
    table_nulos = getNulos(FULL_dataset[key_df])
    nulos_dict[key_df] = table_nulos
  return nulos_dict

In [5]:
getFULLNulos(FULL_dataset)

{'Abandono_ESO':           column_name  percent_missing
 SEXO             SEXO         0.000000
 HATR_BSQ1   HATR_BSQ1         0.000000
 HATR_F15     HATR_F15         0.000000
 HATR_F14     HATR_F14         0.000000
 HA_TRAB       HA_TRAB         0.000000
 ...               ...              ...
 DIAS19         DIAS19        60.410095
 DIAS18         DIAS18        68.717140
 DIAS17         DIAS17        81.177708
 DIAS16         DIAS16        90.694006
 DIAS15         DIAS15        96.161935
 
 [310 rows x 2 columns], 'BACH':           column_name  percent_missing
 SEXO             SEXO         0.000000
 HATR_BSQ4   HATR_BSQ4         0.000000
 HATR_BSQ3   HATR_BSQ3         0.000000
 HATR_BSQ2   HATR_BSQ2         0.000000
 HATR_BSQ1   HATR_BSQ1         0.000000
 ...               ...              ...
 DIAS19         DIAS19        61.798172
 DIAS18         DIAS18        75.228502
 DIAS17         DIAS17        82.727103
 DIAS16         DIAS16        90.244357
 DIAS15         DIAS15        

In [6]:
for key_df in FULL_dataset:
  FULL_dataset[key_df].replace([' '], np.nan, inplace=True)


In [7]:
getFULLNulos(FULL_dataset)

{'Abandono_ESO':          column_name  percent_missing
 SEXO            SEXO         0.000000
 TIC              TIC         0.000000
 C8_1            C8_1         0.000000
 C8_2            C8_2         0.000000
 C8_3            C8_3         0.000000
 ...              ...              ...
 ESTA_B19    ESTA_B19        99.947424
 DURCUR5      DURCUR5       100.000000
 RESCUR5      RESCUR5       100.000000
 DURCUR6      DURCUR6       100.000000
 RESCUR6      RESCUR6       100.000000
 
 [310 rows x 2 columns], 'BACH':          column_name  percent_missing
 SEXO            SEXO              0.0
 TRBPRN1      TRBPRN1              0.0
 MOV_D9        MOV_D9              0.0
 MOVPAI3      MOVPAI3              0.0
 MOVPAI2      MOVPAI2              0.0
 ...              ...              ...
 EST_B14A    EST_B14A            100.0
 EST_B38A    EST_B38A            100.0
 DURCUR6      DURCUR6            100.0
 RESCUR6      RESCUR6            100.0
 EST_B49A    EST_B49A            100.0
 
 [303 rows x

### Eliminamos aquellas columnas con un valor de nulos superior al 60% y asignamos -1 a los valores nulos para evitar errores al crear las tablas del DW

In [8]:
def removeNulos(df, name):
  num_col_rem = 0
  tabla_nulos = getNulos(df)
  for i in tabla_nulos.itertuples(): 
    if i.percent_missing > 80.0:
      del df[i.column_name]
      num_col_rem +=1
  print('Numero de columnas eliminadas en {}: {}'.format(name, num_col_rem))

for key_df in FULL_dataset:
  removeNulos(FULL_dataset[key_df], key_df)
  FULL_dataset[key_df].fillna(-1, inplace=True)

Numero de columnas eliminadas en Abandono_ESO: 111
Numero de columnas eliminadas en ESO: 116
Numero de columnas eliminadas en BACH: 104
Numero de columnas eliminadas en GradoMedio: 119
Numero de columnas eliminadas en GradoSuperior: 112


In [9]:
getFULLNulos(FULL_dataset)

{'Abandono_ESO':          column_name  percent_missing
 SEXO            SEXO              0.0
 TR_E21_2    TR_E21_2              0.0
 TR_E21_3    TR_E21_3              0.0
 TR_E21_4    TR_E21_4              0.0
 TR_E21_5    TR_E21_5              0.0
 ...              ...              ...
 IDIOMA2      IDIOMA2              0.0
 NIV_ID2      NIV_ID2              0.0
 ACREIDI2    ACREIDI2              0.0
 IDIOMA4      IDIOMA4              0.0
 FACTOR        FACTOR              0.0
 
 [199 rows x 2 columns], 'BACH':          column_name  percent_missing
 SEXO            SEXO              0.0
 TR_E21_3    TR_E21_3              0.0
 TR_E21_4    TR_E21_4              0.0
 TR_E21_5    TR_E21_5              0.0
 TR_E21_6    TR_E21_6              0.0
 ...              ...              ...
 NIV_ID2      NIV_ID2              0.0
 ACREIDI2    ACREIDI2              0.0
 IDIOMA3      IDIOMA3              0.0
 TITIDI        TITIDI              0.0
 FACTOR        FACTOR              0.0
 
 [199 rows x

## Seleccionamos las columnas que nos sean utiles para la construccion del DW

In [10]:
def newDF(df, arr_cols):
  new_dict = {}
  for col_name in arr_cols:
    try:
      new_dict[col_name] = df[col_name]
    except Exception as e:
      pass
  return pd.DataFrame.from_dict(new_dict)



In [11]:
AESO_final_columns = ['SEXO','ANIO_NAC','NACIO','PAIS_NACI','T_CENTRO','CCAA','DISCA','CA_RES','ESTUDIOS_PADRE','ESTUDIOS_MADRE','ESTA_B2','ESTA_B5','ESTA_B7','EST_B8','EST_B9','EST_B10','EST_B11','EST_B12B_1','EST_B21','EST_B23B_1','EST_B23B_2','EST_B28','EST_B33','EST_B35B_1','EST_B35B_2','EST_B40','EST_B45','EST_B47B_1','EST_B47B_2','EST_B52','EST_B56','EST_B58B_1','EST_B58B_2','EST_B63','EST_B67','EST_B69B_1','EST_B69B_2','IDIOMAS','IDIOMA1','NIV_ID1','ACREIDI1','IDIOMA2','NIV_ID2','ACREIDI2','IDIOMA3','NIV_ID3','ACREIDI3','IDIOMA4','NIV_ID4','ACREIDI4','TITIDI','TIC','C8_5','C8_6','C8_7','MVFUERA','MOVPAI1','MOVPAI2','MOVPAI3','MOVRA21','MOVRA22','MOVRA23','MOVRA24','MOVRA25','MOVRA26','MOV_D7','RETORNO','TRBPRN1','TRBPRN2','TRINAC_E4','TRINAC_E5','TRINAC_E6','TRAC_E9','LUG_TRAB_PAIS','CATRAB','SIT_PRO','JORNADA','TRABOC','TR_CNAE','TR_ANIO','TR_SUELDO','TR_E19','HL_F6','HL_F8','HL_JOR','HL_F10','HL_F11','HA_TRAB','HATR_F14','P_CONTI','HATR_TIEMP','HATR_NUM','NOTRAB_BUS','NOTRAB_BUSM','NOTRAB_BUST','HO_PAREJ','HO_NPER','HO_NPER_TR','HO_TIPO','N_CURS']
ESO_final_columns = ['SEXO','JORNADA','ANIO_NAC','NACIO','PAIS_NACI','T_CENTRO','CCAA','DISCA','LUG_RES_PAIS','CA_RES','ESTUDIOS_PADRE','ESTUDIOS_MADRE','EST_B2','EST_B3B_1','EST_B7','EST_B14B_1','EST_B14B_2','EST_B19','EST_B24','EST_B26B_1','EST_B26B_2','EST_B31','EST_B36','EST_B38B_1','EST_B38B_2','EST_B43','EST_B47','EST_B49B_1','EST_B49B_2','EST_B53T','EST_B57','EST_B59B_1','EST_B59B_2','IDIOMAS','TITIDI','TIC','MOV_D7','MOV_D9','TRBPRN1','TRBPRN2','TRAC_E9','SIT_PRO','JORNADA','TR_CNAE','TR_ANIO','TR_SUELDO','TR_E22','HL_F6','HA_TRAB','HATR_F14','P_CONTI','PR_SIT_PR','PR_JOR','PR_FORM','PR_SUELDO','HATR_TIEMP','NOTRAB_BUS','HO_G1','HO_PAREJ','HO_NPER','HO_NPER_TR','HO_TIPO','N_CURS']
BACH_final_columns = ['SEXO','ANIO_NAC','NACIO','NACIO1','NACIO2','PAIS_NACI','T_CENTRO','CCAA','MOD_BACH','DISCA','PROXY','LUG_RES_PAIS','CA_RES','PAIS_NAC_PADRE','ESTUDIOS_PADRE','PAIS_NAC_MADRE','ESTUDIOS_MADRE','EST_B1','EST_B2','EST_B3A','EST_B3B_1','EST_B3B_2','ESTA_B9','ESTA_B10','ESTA_B11','EST_B12','EST_B14A','EST_B14B_1','EST_B14B_2','EST_B14C','EST_B16','EST_B19','ESTA_B21','ESTA_B22','ESTA_B23','EST_B24','EST_B26A','EST_B26B_1','EST_B26B_2','EST_B28','EST_B31','ESTA_B33','ESTA_B34','ESTA_B35','EST_B36','EST_B38A','EST_B38B_1','EST_B38B_2','EST_B40','EST_B43','ESTA_B45','ESTA_B46','ESTA_B46B','EST_B47','EST_B49A','EST_B49B_1','EST_B49B_2','EST_B51','EST_B53T','ESTA_B55','ESTA_B56','EST_B57','EST_B59A','EST_B59B_1','EST_B59B_2','EST_B61','C2_4','C2_6','IDIOMAS','IDIOMA1','NIV_ID1','ACREIDI1','IDIOMA3','NIV_ID3','ACREIDI3','NIV_ID4','ACREIDI4','TIC','C9_1','C9_2','C9_3','C9_4','C9_6','MVFUERA','MOV_D7','RETORNO','TRBPRN1','TRINAC_E5','TRINAC_E6','TRPARA_E5','TRPARA_E6','TRPARA_E7_1','TRPARA_E7_2','TRPARA_E7_3','TRPARA_E7_4','TRPARA_E7_5','TRPARA_E7_6','TRPARA_E7_7','TRPARA_E7_8','TRPARA_E7_9','TRPARA_E8_1','TRPARA_E8_2','TRPARA_E8_3','TRPARA_E8_4','TRPARA_E8_5','TRPARA_E8_6','TRPARA_E8_7','TRPARA_E8_8','TRAC_E9','LUG_TRAB_PAIS','SIT_PRO','JORNADA','TRABOC','TR_CNAE','TR_ANIO','TR_SUELDO','TR_E19','TR_E22','HL_F6','HL_F8','HL_JOR','HL_F10','HL_F11','HA_TRAB','HATR_F14','HATR_F15','P_CONTI','PR_OCU','PR_JOR','PR_NIVEL','PR_SUELDO','HATR_TIEMP','NOTRAB_BUS','NOTRAB_BUSM','NOTRAB_BUST','NOTRAB_BSQ1','NOTRAB_BSQ2','NOTRAB_BSQ3','NOTRAB_BSQ4','NOTRAB_BSQ5','NOTRAB_BSQ6','NOTRAB_BSQ7','NOTRAB_BSQ8','NOTRAB_BSQ9','NOTRAB_DIF1','NOTRAB_DIF2','NOTRAB_DIF3','NOTRAB_DIF4','NOTRAB_DIF5','NOTRAB_DIF6','NOTRAB_DIF7','NOTRAB_DIF8','RECHA_TR','HO_G1','HO_PAREJ','HO_NPER','HO_NPER_TR','HO_TIPO','COTIZ15','DIAS15','BCOTC15','COTIZ16','DIAS16','BCOTC16','COTIZ17','DIAS17','BCOTC17','N_CURS']
GM_final_columns = ['SEXO','ANIO_NAC','NACIO','PAIS_NACI','T_CENTRO','CCAA','DISCA','LUG_RES_PAIS','CA_RES','ESTUDIOS_PADRE','ESTUDIOS_MADRE','EST_B0A','EST_B2','EST_B3C','EST_B5','EST_B12','EST_B14C','EST_B26C','EST_B36','EST_B38C','EST_B47','EST_B49C','EST_B59C','IDIOMAS','MOV_D7','TRBPRN1','TRAC_E9','LUG_TRAB_PAIS','SIT_PRO','JORNADA','TR_CNAE','TR_ANIO','TR_SUELDO','TR_E22','HL_F1','HL_F2','HL_F6','HL_F8','HL_JOR','HA_TRAB','HATR_F14','P_CONTI','PR_F20','PR_JOR','PR_SUELDO','HATR_TIEMP','HATR_NUM','RECHA_TR','HO_G1','HO_PAREJ','HO_NPER','HO_NPER_TR','HO_TIPO','N_CURS']
GS_final_columns = ['SEXO','ANIO_NAC','NACIO','PAIS_NACI','T_CENTRO','CCAA','GRUPO_ACT','DISCA','LUG_RES_PAIS','CA_RES','ESTUDIOS_PADRE','ESTUDIOS_MADRE','EST_B1','EST_B4','EST_B8_1','EST_B8_2','EST_B8_3','EST_B8_4','EST_B8_5','EST_B8_6','EST_B8_7','EST_B8_8','EST_B11_1','EST_B17','EST_B28','IDIOMAS','TITIDI','TIC','C8_2','MOV_D7','MOV_D9','TRBPRN1','TRAC_E9','SIT_PRO','JORNADA','TR_ANIO','TR_SUELDO','TR_E22','HL_F1','HL_F2','HL_F3','HL_F6','HL_F8','HL_JOR','HL_F11','HA_TRAB','P_CONTI','PR_F20','PR_JOR','PR_SUELDO','HATR_TIEMP','HO_G1','HO_PAREJ','HO_NPER','HO_NPER_TR','HO_TIPO']

selected_columns = {
    'Abandono_ESO':AESO_final_columns,
    'ESO':ESO_final_columns,
    'BACH':BACH_final_columns,
    'GradoMedio':GM_final_columns,
    'GradoSuperior':GS_final_columns,
}


In [12]:
FULL_dataset_final = {}

#Esto se puede hacer porque ambos diccionarios son iguales y sus llaves son las mismas, no muy recomendable pero bueno
for common_key in FULL_dataset:
  FULL_dataset_final[common_key] = newDF(FULL_dataset[common_key], selected_columns[common_key])

In [13]:
FULL_dataset_final

{'Abandono_ESO':        SEXO  ANIO_NAC  NACIO  PAIS_NACI  ...  HO_NPER  HO_NPER_TR  HO_TIPO  N_CURS
 IDENT                                    ...                                      
 1         1      1995      1          1  ...        2           1        3       0
 2         2      1995      1          1  ...        3           1        3       0
 3         1      1996      1          3  ...        3           0        6       0
 4         1      1996      1          1  ...        3           1        6       0
 5         2      1996      1          1  ...        5           3        5       0
 ...     ...       ...    ...        ...  ...      ...         ...      ...     ...
 1898      1      1998      1          1  ...        4           2        7       0
 1899      2      1998      1          1  ...        3           0        6       1
 1900      1      1996      1          1  ...        9           9        9       0
 1901      1      1996      2          3  ...        9      

# Extraccion y transformacion de los datos

In [14]:
for key in FULL_dataset_final:
  print(key)
  print(FULL_dataset_final[key].columns)

Abandono_ESO
Index(['SEXO', 'ANIO_NAC', 'NACIO', 'PAIS_NACI', 'T_CENTRO', 'CCAA', 'DISCA',
       'CA_RES', 'ESTUDIOS_PADRE', 'ESTUDIOS_MADRE', 'ESTA_B2', 'ESTA_B5',
       'ESTA_B7', 'EST_B11', 'EST_B12B_1', 'EST_B21', 'EST_B23B_1',
       'EST_B23B_2', 'EST_B28', 'EST_B33', 'EST_B35B_1', 'EST_B35B_2',
       'EST_B40', 'EST_B45', 'EST_B47B_1', 'EST_B47B_2', 'EST_B52', 'EST_B56',
       'EST_B58B_1', 'EST_B58B_2', 'EST_B63', 'EST_B67', 'EST_B69B_1',
       'EST_B69B_2', 'IDIOMAS', 'IDIOMA1', 'NIV_ID1', 'ACREIDI1', 'IDIOMA2',
       'NIV_ID2', 'ACREIDI2', 'IDIOMA3', 'IDIOMA4', 'TIC', 'C8_5', 'C8_6',
       'C8_7', 'MVFUERA', 'MOVPAI1', 'MOVPAI2', 'MOVPAI3', 'MOV_D7', 'TRBPRN1',
       'TRBPRN2', 'TRINAC_E4', 'TRAC_E9', 'LUG_TRAB_PAIS', 'CATRAB', 'SIT_PRO',
       'JORNADA', 'TRABOC', 'TR_CNAE', 'TR_ANIO', 'TR_SUELDO', 'TR_E19',
       'HL_F6', 'HA_TRAB', 'HATR_F14', 'P_CONTI', 'HATR_TIEMP', 'HATR_NUM',
       'HO_PAREJ', 'HO_NPER', 'HO_NPER_TR', 'HO_TIPO', 'N_CURS'],
      dtype='objec

# Formacion de los datasets previos al DW
Una vez tenemos todos los datos consistentes y limpios, se diseña el esquema del DW para poder traspasar los datos de todos los datasets a nuevos datasets los cuales representaran cada una de las tabla que compondran el DW. 

En nuestro caso tendremos un DS Fact_Transicion, el cual sera nuestra tabla de hechos principal, y estara rodeada de las dimensiones de Persona, Estudios, trabajo, movilidad e idiomas.


La tabla de hechos contendra las PK de las tablas de dimension, asi como el tipo de estudio de la persona (ESO, BACH ...) y si su situacion laboral a fecha de la realizacion de la encuesta.

instrucciones:

recorrer uno por uno los datasets
instanciar codificacion de tipo de estudio
  recorrer cada persona
  instanciar PK_Persona
  añadir a Tabla_Persona los datos y PK

  instruccion final debe ser añadir a la tabla de hechos para tener generadas todas la PK


In [15]:


tabla_hechos = pd.DataFrame(columns=['ID_Pers', 'ID_Estudios', 'ID_Trabajo', 'ID_Idioma','ID_Movi', 'cod_estudio', 'situ_laboral'])

dimension_persona = pd.DataFrame(columns=['ID_Pers', 'sexo', 'ano_nac', 'nacion'])
dimension_estudios = pd.DataFrame(columns=['ID_Estud', 'est_padre', 'est_madre'])
dimension_trabajo = pd.DataFrame(columns=['ID_Trab', 'jornada'])
dimension_movilidad = pd.DataFrame(columns=['ID_Movi', 'movil'])
dimension_idioma = pd.DataFrame(columns=['ID_Idiom', 'idomas'])



PK_persona = 0
PK_estudios = 0
PK_trabajo = 0
PK_movilidad = 0
PK_idioma = 0
codigo_estudio = 0

for df_key in FULL_dataset_final:
  for index, row in FULL_dataset_final[df_key].iterrows():
    # ---- Dimension persona
    try:
      new_row_persona = [PK_persona, row.SEXO, row.ANIO_NAC, row.NACIO]
      dimension_persona.loc[len(dimension_persona)] = new_row_persona
      PK_persona +=1
    except:
      pass
    # ---- Dimension estudios
    try:
      new_row_estudios = [PK_estudios,  row.ESTUDIOS_PADRE, row.ESTUDIOS_MADRE]
      dimension_estudios.loc[len(dimension_estudios)] = new_row_estudios
      PK_estudios +=1
    except:
      pass
    # ---- Dimension trabajo
    try:
      new_row_trabajo = [PK_trabajo, row.JORNADA]
      dimension_trabajo.loc[len(dimension_trabajo)] = new_row_trabajo
      PK_trabajo +=1
    except:
      pass
    # ---- Dimension movilidad
    try:
      new_row_movilidad = [PK_movilidad, row.MOV_D7]
      dimension_movilidad.loc[len(dimension_movilidad)] = new_row_movilidad
      PK_movilidad +=1
    except:
      pass
    # ---- Dimension idioma
    try:
      new_row_idioma = [PK_idioma, row.IDIOMAS]
      dimension_idioma.loc[len(dimension_idioma)] = new_row_idioma
      PK_idioma +=1
    except:
      pass
    # ---- tabla_hechos
    new_row_tabla_hechos = [PK_persona, PK_estudios, PK_trabajo, PK_movilidad, PK_idioma, codigo_estudio, row.TRBPRN1]
    tabla_hechos.loc[len(tabla_hechos)] = new_row_tabla_hechos
  print(PK_persona)

  codigo_estudio +=1

%time


1902
8166
13527
19270
27072
CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.44 µs


In [19]:
tabla_hechos

,ID_Pers,ID_Estudios,ID_Trabajo,ID_Idioma,ID_Movi,cod_estudio,situ_laboral
0,1,1,1,1,1,0,3
1,2,2,2,2,2,0,3
2,3,3,3,3,3,0,3
3,4,4,4,4,4,0,1
4,5,5,5,5,5,0,1
...,...,...,...,...,...,...,...
27067,27068,27068,27068,27068,27068,0,3
27068,27069,27069,27069,27069,27069,0,1
27069,27070,27070,27070,27070,27070,0,2
27070,27071,27071,27071,27071,27071,0,1


# Carga de datos en el datawarehouse

In [25]:
from google.colab import drive
drive.mount('/content/drive')


tabla_hechos.to_csv('./tabla_hechos.csv')
dimension_persona.to_csv('./dimension_persona.csv')
dimension_estudios.to_csv('./dimension_estudios.csv')
dimension_trabajo.to_csv('./dimension_trabajo.csv')
dimension_movilidad.to_csv('./dimension_movilidad.csv')
dimension_idioma.to_csv('./dimension_idioma.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
